In [ ]:
from erddaplogs.logparse import ErddapLogParser

# Instantiate parser and set verbosity

(False to disable print messages)

In [ ]:
parser = ErddapLogParser()
parser.verbose=True

### set path to apache and/or nginx files

In [ ]:
#apache_logs_dir = "path/to/your/apache/logs/"
nginx_logs_dir = "../example_data/nginx_example_logs/"

### load log files

Can call just apache or just nginx as needed

In [ ]:
#parser.load_apache_logs(apache_logs_dir)
parser.load_nginx_logs(nginx_logs_dir)


**optional** subset datasetframe to desired number of rows, evenly spaced in time. Useful for tuning/testing

In [ ]:
#parser.subset_df(rows=1000) 

In [ ]:
parser.filter_non_erddap()
parser.filter_spam()
parser.filter_locales()
parser.filter_user_agents()
parser.filter_common_strings()

# get info on ip addresses

In [ ]:
parser.get_ip_info(download_new=True)
parser.filter_organisations()


### Optional: read in datasets.xml

This adds another column of data: the dataset type in ERDDAP

In [ ]:
parser.parse_datasets_xml("../example_data/datasets.xml")

# Parse string data for additional columns

In [ ]:
parser.parse_columns()

# Get requests dataframe

In [ ]:
df = parser.df
df.head()

You can also check out the ip dataframe

In [ ]:
df_ip = parser.ip
df_ip.head()

# Plotting the results

This additional functionality requires several extra packages



In [ ]:
import erddaplogs.plot_functions as plot_functions

Plot requests summed over a user specified number of days

In [ ]:
plot_functions.plot_daily_requests(df, num_days=1)

Plot the total size of requests summed over a number of days

In [ ]:
plot_functions.plot_bytes(df)

Plot the most popular values for chosen columns. This function returns a dataframe of the top results

In [ ]:
plot_functions.plot_most_popular(df, col_name='dataset_id');

The ERDDAP request type will be one of: griddap, tabledap, info, files, unknown

In [ ]:
plot_functions.plot_most_popular(df, col_name='erddap_request_type');

In [ ]:
plot_functions.plot_most_popular(df, col_name='user_agent', rows=20);

In [ ]:
plot_functions.plot_most_popular(df, col_name='user_agent', rows=5);

Create a map of the geolocations of the users that have made the most requests. We use the subnet (first 3 of dotted quad of the ipv4 address) to try to aggregate requests from e.g. a single building/insitution

In [ ]:
plot_functions.plot_map_requests(df, aggregate_on='ip_subnet');

Get a list of the ip adddresses that made the most requests and make some more in depth plots for them

In [ ]:
dfa = plot_functions.plot_most_popular(df, col_name='ip', rows=20)

In [ ]:
import polars as pl

for rank, ip in enumerate(dfa['ip'].to_list()):
    df_sub = df.filter(pl.col('ip')==ip)
    plot_functions.plot_for_single_ip(df_sub, f'visitor_rank_{rank}_ip_{ip}')

# Exporting Results

In order to export anonymized and aggregated results, simply call:

In [ ]:
parser.export_data()